In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from invert.cmaps import parula
pp = dict(surface='white', hemi='both')

In [3]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
leadfield -= leadfield.mean(axis=0)

n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.1s remaining:    2.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


# Simulate

In [117]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=1, extents=(24, 25), duration_of_trial=0.1, target_snr=1e99)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
y = evoked.data
x = stc.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 12.46it/s]

Using control points [0.00000000e+00 0.00000000e+00 1.94289228e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# MUSIC

In [118]:
# Parameters
n = 6
stop_crit = 0.95

leadfield -= leadfield.mean(axis=0)

_, n_time = y.shape
C = y@y.T
U, D, _  = np.linalg.svd(C, full_matrices=False)
Us = U[:, :n]
Ps = Us@Us.T

mu = np.zeros(n_dipoles)
for p in range(n_dipoles):
    l = leadfield[:, p][:, np.newaxis]
    norm_1 = np.linalg.norm(Ps @ l)
    norm_2 = np.linalg.norm(l)
    mu[p] = norm_1 / norm_2
mu[mu<stop_crit] = 0

dipole_idc = np.where(mu!=0)[0]
x_hat = np.zeros((n_dipoles, n_time))
x_hat[dipole_idc, :] = np.linalg.pinv(leadfield[:, dipole_idc]) @ y

# Plot

In [119]:
stc_ = stc.copy()
stc_.data = x_hat
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, 'MUSIC', 'title',
               font_size=14)

Using control points [0.00000000e+00 0.00000000e+00 4.05791422e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# RAP-MUSIC

In [120]:
# Parameters
n = 10
k = 3
stop_crit = 0.95
alpha = 0

leadfield -= leadfield.mean(axis=0)

I = np.identity(n_chans)
C = y @ y.T + alpha * I

dipole_idc = []
B = None
n_time = y.shape[1]
for i in range(k):
    print(i)
    U, D, _= np.linalg.svd(C, full_matrices=False)
    Us = U[:, :n]
    Ps = Us@Us.T

    mu = np.zeros(n_dipoles)
    for p in range(n_dipoles):
        l = leadfield[:, p][:, np.newaxis]
        norm_1 = np.linalg.norm(Ps @ l)
        norm_2 = np.linalg.norm(l)
        mu[p] = norm_1 / norm_2
    dipole_idx = np.argmax(mu)
    dipole_idc.append( dipole_idx )

    if np.max(mu) < stop_crit:
        print("breaking")
        break

    if i == 0:
        B = leadfield[:, dipole_idx][:, np.newaxis]
    else:
        B = np.hstack([B, leadfield[:, dipole_idx][:, np.newaxis]])
    
    Q = I - B @ np.linalg.pinv(B)
    C = Q @ Us

dipole_idc = np.array(dipole_idc)
x_hat = np.zeros((n_dipoles, n_time))
x_hat[dipole_idc, :] = np.linalg.pinv(leadfield[:, dipole_idc]) @ y

0
1
breaking


# Plot

In [121]:
# stc_ = stc.copy()
# stc_.data[dipole_idc, 0] = 1
# brain = stc_.plot(**pp)
# brain.add_text(0.1, 0.9, 'RAP-MUSIC', 'title',
#                font_size=14)

stc_ = stc.copy()
stc_.data = x_hat
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, 'RAP-MUSIC estimate', 'title',
               font_size=14)

Using control points [0.0000000e+00 0.0000000e+00 1.9587626e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# TRAP-MUSIC

In [122]:
# Parameters
n = 10
k = 3
stop_crit = 0.95
alpha  = 0

leadfield -= leadfield.mean(axis=0)

I = np.identity(n_chans)
C = y @ y.T + alpha * I

dipole_idc = []
B = None
n_time = y.shape[1]
for i in range(k):
    print(i)
    U, D, _= np.linalg.svd(C, full_matrices=False)
    Us = U[:, :n-i]
    Ps = Us@Us.T

    mu = np.zeros(n_dipoles)
    for p in range(n_dipoles):
        l = leadfield[:, p][:, np.newaxis]
        norm_1 = np.linalg.norm(Ps @ l)
        norm_2 = np.linalg.norm(l)
        mu[p] = norm_1 / norm_2
    dipole_idx = np.argmax(mu)
    dipole_idc.append( dipole_idx )

    if np.max(mu) < stop_crit:
        print("breaking")
        break

    if i == 0:
        B = leadfield[:, dipole_idx][:, np.newaxis]
    else:
        B = np.hstack([B, leadfield[:, dipole_idx][:, np.newaxis]])
    
    Q = I - B @ np.linalg.pinv(B)
    C = Q @ Us

dipole_idc = np.array(dipole_idc)
x_hat = np.zeros((n_dipoles, n_time))
x_hat[dipole_idc, :] = np.linalg.pinv(leadfield[:, dipole_idc]) @ y

0
1
breaking


In [123]:
# stc_ = stc.copy()
# stc_.data[dipole_idc, 0] = 1
# brain = stc_.plot(**pp)
# brain.add_text(0.1, 0.9, 'RAP-MUSIC', 'title',
#                font_size=14)

stc_ = stc.copy()
stc_.data = x_hat
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, 'TRAP-MUSIC estimate', 'title',
               font_size=14)

Using control points [0.00000000e+00 0.00000000e+00 2.02906024e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
